In [2]:
import numpy as np
import pickle
import cartopy.io.shapereader as shpreader
from cartopy.feature import ShapelyFeature
from shapely.prepared import prep
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd

In [11]:
# some useful arrays to translate between naming conventions

election_year_list = np.array([1992, 1994, 1996, 1998, 2000, 2002, 2004, 2006, 2008, 2010, 2012, 
                                   2014, 2016, 2018])
congress_ID_list = np.array([103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116])

state_names = np.array(['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 
               'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 
               'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 
               'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 
               'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 
               'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE', 'NEW JERSEY', 'NEW MEXICO', 
               'NEW YORK', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 
               'OREGON', 'PENNSYLVANIA', 'RHODE ISLAND', 'SOUTH CAROLINA', 
               'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 
               'VIRGINIA', 'WASHINGTON', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING'])

state_abbrs = np.array(['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL',
              'IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT',
              'NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI',
              'SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY'])

state_fips = np.array([1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 
                       24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 
                       41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56])

In [4]:
# The proj.4 string:
# +proj=longlat +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +no_defs

In [5]:
year = 2014
congress_ID = '114'
shpfilename = 'districtShapes-{0}/districts{0}.shp'.format(congress_ID)

In [16]:
election_year=2016
verbose = True
# convert election year to "Nth Congress" 
congress_ID = congress_ID_list[election_year_list==election_year][0]
district_df = pickle.load(open('../Datasets/master_index.p','rb'))
district_df['shape'] = [np.nan]*district_df.shape[0] # make a blank column
district_df['shape'] = district_df['shape'].astype(object) # reassign to object so it can hold shapely stuff



AK_01_2016 was read in.
DE_01_2016 was read in.
MT_01_2016 was read in.
ND_01_2016 was read in.
SD_01_2016 was read in.
VT_01_2016 was read in.
WY_01_2016 was read in.
AL_01_2016 was read in.
AZ_01_2016 was read in.
AR_01_2016 was read in.
CA_01_2016 was read in.
CO_01_2016 was read in.
CT_01_2016 was read in.
FL_01_2016 was read in.
GA_01_2016 was read in.
HI_01_2016 was read in.
ID_01_2016 was read in.
IL_01_2016 was read in.
IN_01_2016 was read in.
IA_01_2016 was read in.
KS_01_2016 was read in.
KY_01_2016 was read in.
LA_01_2016 was read in.
ME_01_2016 was read in.
MD_01_2016 was read in.
MA_01_2016 was read in.
MI_01_2016 was read in.
MN_01_2016 was read in.
MS_01_2016 was read in.
MO_01_2016 was read in.
NE_01_2016 was read in.
NV_01_2016 was read in.
NH_01_2016 was read in.
NJ_01_2016 was read in.
NM_01_2016 was read in.
NY_01_2016 was read in.
NC_01_2016 was read in.
OH_01_2016 was read in.
OK_01_2016 was read in.
OR_01_2016 was read in.
PA_01_2016 was read in.
RI_01_2016 was r

PA_15_2016 was read in.
TX_15_2016 was read in.
CA_16_2016 was read in.
FL_16_2016 was read in.
IL_16_2016 was read in.
NY_16_2016 was read in.
OH_16_2016 was read in.
PA_16_2016 was read in.
TX_16_2016 was read in.
CA_17_2016 was read in.
FL_17_2016 was read in.
IL_17_2016 was read in.
NY_17_2016 was read in.
PA_17_2016 was read in.
TX_17_2016 was read in.
CA_18_2016 was read in.
FL_18_2016 was read in.
IL_18_2016 was read in.
NY_18_2016 was read in.
PA_18_2016 was read in.
TX_18_2016 was read in.
CA_19_2016 was read in.
FL_19_2016 was read in.
NY_19_2016 was read in.
TX_19_2016 was read in.
CA_20_2016 was read in.
FL_20_2016 was read in.
NY_20_2016 was read in.
TX_20_2016 was read in.
CA_21_2016 was read in.
FL_21_2016 was read in.
NY_21_2016 was read in.
TX_21_2016 was read in.
CA_22_2016 was read in.
FL_22_2016 was read in.
NY_22_2016 was read in.
TX_22_2016 was read in.
CA_23_2016 was read in.
FL_23_2016 was read in.
NY_23_2016 was read in.
TX_23_2016 was read in.
CA_24_2016 was r

In [8]:
d= next(districts)

In [9]:
d

<Record: <shapely.geometry.multipolygon.MultiPolygon object at 0x11a7bfb00>, {'STATEFP': '02', 'CD115FP': '00', 'GEOID': '0200', 'NAMELSAD': 'Congressional District (at Large)', 'LSAD': 'C1', 'CDSESSN': '115', 'MTFCC': 'G5200', 'FUNCSTAT': 'N', 'ALAND': 1477946266785, 'AWATER': 245390495931, 'INTPTLAT': '+63.3461909', 'INTPTLON': '-152.8370690'}, <fields>>

In [ ]:
def read_shapefiles(election_years,verbose=True):
    # read in the standard dictionary 
    district_df = pickle.load(open('../Datasets/master_index.p','rb'))
    district_df['shape'] = [np.nan]*district_df.shape[0] # make a blank column
    district_df['shape'] = district_df['shape'].astype(object) # reassign to object so it can hold shapely stuff

    for election_year in election_years:
        # convert election year to "Nth Congress" 
        congress_ID = congress_ID_list[election_year_list==election_year][0]

        # read in the shapefile (must be named 'districtsN.shp' in a folder titled 'districtShapesN')
        shpfilename = 'districtShapes-{0}/districts{0}.shp'.format(congress_ID)
        reader = shpreader.Reader(shpfilename) 
        districts = reader.records() # get full records
        geometries = reader.geometries() # get just the shape

        # pre-2016 files come from http://cdmaps.polisci.ucla.edu/
        # they are a bit cleaner
        if election_year < 2016:
            # put the shapefiles into the standard dictionary
            for record in reader.records(): # loop over districts
                attr = record.attributes # dictionary of information about the district
                poly = record.geometry # coordinates of the district as a shapely polygon

                # 1) get the state abbr of the district
                if any(state_names==attr['STATENAME'].upper()): # filter out districts that aren't in states
                    ST = state_abbrs[state_names==attr['STATENAME'].upper()][0]
                else: # pretty much just Washington, DC
                    print('{} is not a state.'.format(attr['STATENAME'].upper()))
                    continue
                # 2) get the id of the district
                id_int = int(attr['DISTRICT'])
                if id_int == 0: # todo: do we want to change this back? Looks like 0 for at-large districts is the convention. 
                    id_int = 1
                ID = '{0:02d}'.format(id_int)
                # 3) put it all together into an index
                ind = '{}_{}_{}'.format(ST, ID, election_year)
                if verbose:
                    print('{} was read in.'.format(ind))
                # put the polygon in the dictionary
                district_df.at[ind,'shape'] = poly
            
        # 2016 and later are US Census Bureau Tiger Line Files 
        # (and they're a bit messier)
        if election_year >= 2016:
            # read in the shapefile (must be named 'districtsN.shp' in a folder titled 'districtShapesN')
            shpfilename = 'tl_2016_us_cd{0}/tl_{1}_us_cd{0}.shp'.format(congress_ID,election_year)
            reader = shpreader.Reader(shpfilename) 
            districts = reader.records() # get full records
            geometries = reader.geometries() # get just the shape

            # put the shapefiles into the standard dictionary
            for record in reader.records(): # loop over districts
                attr = record.attributes # dictionary of information about the district
                poly = record.geometry # coordinates of the district as a shapely polygon

                # 1) get the state abbr of the district
                if any(state_fips==int(attr['STATEFP'])): # filter out districts that aren't in states
                    ST = state_abbrs[state_fips==int(attr['STATEFP'])][0]
                else: # pretty much just Washington, DC
                    print('{} is not a state.'.format(int(attr['STATEFP'])))
                    continue
                # 2) get the id of the district
                if (attr['CD{}FP'.format(congress_ID)]=='ZZ'):
                    print('District ID is ZZ, which indicates this is not a congressional district.')
                    continue
                else:
                    id_int = int(attr['CD{}FP'.format(congress_ID)])
                if id_int == 0: # change at-large district ID from 0 to 1 to play nice with our indexing convention.
                    # todo: do we want to change this back? Looks like 0 for at-large districts is the convention. 
                    id_int = 1
                ID = '{0:02d}'.format(id_int)
                # 3) put it all together into an index
                ind = '{}_{}_{}'.format(ST, ID, election_year)
                if verbose:
                    print('{} was read in.'.format(ind))
                # put the polygon in the dictionary
                district_df.at[ind,'shape'] = poly
    
    return district_df

In [ ]:
# find if a district has changed between last year and this year
def check_if_districts_changed(this_year, district_df, threshold_for_change=0.1):
    # loop over states so you only have to compare districts in-state
    # otherwise, comparing each district to 434 other districts would be super slow
    for ST in state_abbrs: 
        prev_year = this_year-2
        # get the relevant districts
        districts = district_df.loc[np.logical_and(district_df['state']==ST,
                                                 district_df['year']==this_year)]
        districts_prev = district_df.loc[np.logical_and(district_df['state']==ST,
                                                 district_df['year']==prev_year)]
        # loop over districts in your current year
        for ind,district in districts.iterrows():
            # find previous year's district
            district_prev = districts_prev.loc[districts_prev['district']==district['district']]
            ind_prev = district_prev.index

            # determine whether district is new or borders have changed
            if ind_prev.shape[0]==0: # if the district didn't exist last year
                # then this district is new this year
                district_df.loc['border_change',ind] = 'new'
            else: 
                # check if the borders have changed
                shape_prev = district_prev['shape'].values[0]
                shape = district['shape']

                # check if shapes intersect with themselves
                if not (shape.is_valid and shape_prev.is_valid): 
                    # if they do, use buffer to correct this
                    print('The following polygons intersected with themselves. Attempting to buffer shape...')
                    if not shape.is_valid:
                        print(ind)
                        shape = shape.buffer(0)
                    if not shape_prev.is_valid:
                        print(ind_prev)
                        shape_prev = shape_prev.buffer(0)

                # calculate overlap percent
                area = shape.area # area of this district
                area_prev = shape_prev.area
                overlap_area = shape.intersection(shape_prev).area # area of overlap between shape and shape_prev
                frac_overlap = overlap_area/np.max([area,area_prev]) # fractional overlap between new and old district
                # divide by the larger of the old or new district
                
                if (1.-frac_overlap) < threshold_for_change: 
                    # then district has not changed
                    district_df.loc[ind,'border_change'] = 'same'
                else:
                    # district has changed
                    district_df.loc[ind,'border_change'] = 'changed'
                    print(ind)
                    print(frac_overlap)
                    
    return district_df

In [ ]:
# compute overlap percent between this district and last year's districts
def district_overlap(this_year, district_df):
    # loop over states so you only have to compare districts in-state
    # otherwise, comparing each district to 434 other districts would be super slow
    for ST in state_abbrs: 
        prev_year = this_year-2
        # get the relevant districts
        districts = district_df.loc[np.logical_and(district_df['state']==ST,
                                                 district_df['year']==this_year)]
        districts_prev = district_df.loc[np.logical_and(district_df['state']==ST,
                                                 district_df['year']==prev_year)]

        for ind,district in districts.iterrows(): # loop over districts in your current year
            overlap_dict = {}
            for ind_prev,district_prev in districts_prev.iterrows(): # loop over districts in previous year
                shape_prev = district_prev['shape']
                shape = district['shape']

                # check if shapes intersect with themselves
                if not (shape.is_valid and shape_prev.is_valid): 
                    # if they do, use buffer to correct this
                    print('The following polygons intersected with themselves. Attempting to buffer shape...')
                    if not shape.is_valid:
                        print(ind)
                        shape = shape.buffer(0)
                    if not shape_prev.is_valid:
                        print(ind_prev)
                        shape_prev = shape_prev.buffer(0)

                # calculate frac overlap
                area = shape.area # area of this district
                area_prev = shape_prev.area
                overlap_area = shape.intersection(shape_prev).area # area of overlap between shape and shape_prev
                frac_overlap = overlap_area/area # fractional overlap between new and old district

                if frac_overlap > 0.:
                    overlap_dict[ind_prev] = frac_overlap

            print(ind)
            print(overlap_dict)
            district_df.at[ind, 'overlap_frac'] = overlap_dict
            
    return district_df

In [ ]:
district_df = read_shapefiles([2010,2012,2014], verbose=False)

In [ ]:
this_year = 2014
district_df = check_if_districts_changed(this_year, district_df, threshold_for_change=0.1)

In [ ]:
this_year = 2012
district_df['overlap_frac'] = [np.nan]*district_df.shape[0] # make a blank column
district_df['overlap_frac'] = district_df['overlap_frac'].astype(object) # convert to object 

overlap_df = district_overlap(this_year, district_df)

In [ ]:
# testing a plot
shape_feature = ShapelyFeature(record.geometry,
                                ccrs.PlateCarree(), edgecolor='black')
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.set_xlim((-125, -67))
ax.set_ylim((24,50))
ax.add_feature(shape_feature, facecolor='blue')
plt.show()